# Flatiron School Data Science Project 4

In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

In [2]:
spark = SparkSession.builder.appName("Movie Recommender").getOrCreate()

25/03/21 11:59:12 WARN Utils: Your hostname, Matthews-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.129.2.122 instead (on interface en0)
25/03/21 11:59:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/21 11:59:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
ratings_df = spark.read.csv("./data/ratings.csv", header=True, inferSchema=True)
ratings_df = ratings_df.repartition(500, "userId")

In [4]:
(training, test) = ratings_df.randomSplit([.8, .2], seed=42)
small_train = training.sample(.01, seed=42)
small_test = test.sample(.02, seed=42)

In [5]:
als = ALS(
    rank=10, maxIter=5, regParam=.1, userCol="userId", itemCol="movieId", ratingCol="rating", coldStartStrategy="drop", numUserBlocks=100, numItemBlocks=100
)

als_model = als.fit(small_train)

25/03/21 12:01:16 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
25/03/21 12:01:18 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [ ]:
als_model.getParam("predictionCol")

Param(parent='ALS_6b51b74a7592', name='predictionCol', doc='prediction column name.')

In [6]:
als_predictions = als_model.transform(small_test)

In [ ]:
als_predictions.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)
 |-- prediction: float (nullable = false)



In [7]:
evaluator = RegressionEvaluator(
    metricName="rmse", 
    labelCol="rating",
    predictionCol="prediction"
)
rmse = evaluator.evaluate(als_predictions)
print(f"RMSE = {rmse}")

RMSE = 3.5300198578628437


In [24]:
user_factors = als_model.userFactors
item_factors = als_model.itemFactors

In [25]:
# Show the learned latent factors
print("User factors:")
user_factors.show(truncate=False)

print("Item factors:")
item_factors.show(truncate=False)

User factors:
+----+------------------------------------------------------------------------------------------------------------------------------+
|id  |features                                                                                                                      |
+----+------------------------------------------------------------------------------------------------------------------------------+
|100 |[1.0727109, 0.046257347, -1.6158141, 1.3239769, -0.5729028, 0.33799928, -0.7600713, 0.12208241, -0.43400073, -0.0906223]      |
|200 |[0.66309077, -0.7346368, -1.890181, 0.76086617, 0.02637443, -0.6290303, -0.26591107, -0.5079492, 0.9851072, 0.2138525]        |
|300 |[1.0776465, -1.6779761, -1.4355997, -0.35906443, -1.0074493, -0.08925376, 0.3475545, -0.6928518, 1.0240145, 0.15007143]       |
|500 |[0.22475551, -1.0283527, -1.9221817, 0.8625367, -1.4161441, -8.1257493E-4, -0.96380794, -0.28952616, 0.5173199, 0.7246166]    |
|600 |[0.40538758, -1.08421, -1.7714063, -1.5594

In [ ]:
user_recs = als_model.recommendForAllUsers(3)

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|    31|[{146264, 4.56496...|
|    53|[{197873, 7.98111...|
|    65|[{197873, 7.85244...|
|    78|[{155565, 6.62584...|
|   108|[{105477, 6.51518...|
|   133|[{118872, 6.05665...|
|   137|[{197873, 8.57042...|
|   148|[{155565, 4.90615...|
|   155|[{118872, 6.11582...|
|   322|[{197873, 7.14091...|
+------+--------------------+
only showing top 10 rows



In [ ]:
user_recs.show(10)

ERROR:root:KeyboardInterrupt while sending command.            (98 + 8) / 10000]
Traceback (most recent call last):
  File "/opt/anaconda3/envs/fis_default/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/opt/anaconda3/envs/fis_default/lib/python3.10/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/opt/anaconda3/envs/fis_default/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 